In [1]:
import sys 
import os 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append('/ocean/projects/cis250068p/jhwang4/idl-project')
import torch as t
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm.auto import tqdm

from transformers import GPTNeoXForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import re
from scipy.spatial.distance import cosine
from src.model import *
from src.data import load_cnn_dataset
#from src.utils import *
from src.utils_batch_v2 import *
#from src.search import *
from src.search_batch_v2 import *


/jet/home/jhwang4/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', '{:.4f}'.format)
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /jet/home/jhwang4/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
DEVICE

'cuda'

In [4]:
# Load the models
fo_model, fo_tokenizer = load_ftfo_model()
ba_model, ba_tokenizer = load_ftba_model()
fo_model = fo_model.to(DEVICE).half()
ba_model = ba_model.to(DEVICE).half()

if t.cuda.device_count() > 1:
    print(t.cuda.device_count())
    fo_model = t.nn.DataParallel(fo_model)
    ba_model = t.nn.DataParallel(ba_model)

fo_model = t.compile(fo_model)
ba_model = t.compile(ba_model)

4


In [5]:
t.cuda.empty_cache()

In [6]:
dataset = load_cnn_dataset(num_samples=15950)
dataset = pd.DataFrame(dataset)

Dataset loaded successfully
Example dataset item: {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places be

In [7]:
exclusion_results = exclusion_search_attribution_search(dataset, fo_model, fo_tokenizer, ba_model, ba_tokenizer, sentence_batch_size=8) # 1m43s, 8m32s 

  0%|                                                                                                | 0/15950 [00:00<?, ?it/s]/ocean/projects/cis250068p/jhwang4/idl-project/src/utils_batch_v2.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with t.no_grad(), autocast():
W0425 02:21:02.436000 37256 torch/_logging/_internal.py:1081] [0/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored
/ocean/projects/cis250068p/jhwang4/idl-project/src/utils_batch_v2.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with t.no_grad(), autocast():
/ocean/projects/cis250068p/jhwang4/idl-project/src/utils_batch_v2.py:176: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with t.no_grad(), autocast():
/jet/home/jhwang4/.local/lib/pytho

In [8]:
EMB_MODEL    = SentenceTransformer('all-MiniLM-L6-v2')
ROUGE_SCORER = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
TFIDF_VEC    = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",
    stop_words=None
)

def calculate_embedding_similarity(highlight, citation):
    he = EMB_MODEL.encode([highlight])[0]
    ce = EMB_MODEL.encode([citation])[0]
    return 1 - cosine(he, ce)

def calculate_rouge_score(highlight, citation):
    return ROUGE_SCORER.score(highlight, citation)['rougeL'].fmeasure

def calculate_tfidf_score(highlight, citation):
    try:
        mat = TFIDF_VEC.fit_transform([highlight, citation]).toarray()
        return 1 - cosine(mat[0], mat[1])
    except ValueError:
        return 0.0

def process_data(data):
    results = []
    for item in data:
        hl = item['highlight']
        rec = {'id': item['id'], 'highlight': hl}
        for citation_type in ['base_citation','fo_citation','ba_citation']:
            prefix = citation_type.split('_')[0]
            cit = item.get(citation_type, None)
            if cit:
                rec[f'{prefix}_emb_similarity'] = calculate_embedding_similarity(hl, cit)
                rec[f'{prefix}_rouge_score']     = calculate_rouge_score(hl, cit)
                rec[f'{prefix}_tfidf_score']     = calculate_tfidf_score(hl, cit)
            else:
                rec[f'{prefix}_emb_similarity'] = None
                rec[f'{prefix}_rouge_score']     = None
                rec[f'{prefix}_tfidf_score']     = None
        results.append(rec)
    return results


exclusion_final_results = process_data(exclusion_results)

/opt/packages/AI/pytorch_23.02-1.13.1-py3/lib/python3.10/site-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [9]:
r = pd.DataFrame(exclusion_final_results) 
r.drop(['id', 'highlight'], axis=1).mean()

base_emb_similarity   0.4774
base_rouge_score      0.1968
base_tfidf_score      0.1759
fo_emb_similarity     0.5984
fo_rouge_score        0.3235
fo_tfidf_score        0.2821
ba_emb_similarity     0.6421
ba_rouge_score        0.3515
ba_tfidf_score        0.3051
dtype: float64

In [10]:
df_exclusion = pd.DataFrame(exclusion_final_results)
mean_series = df_exclusion.drop(['id', 'highlight'], axis=1).mean()

data = {
    'Base_exclusion': [
        mean_series['base_emb_similarity'],
        mean_series['base_rouge_score'],
        mean_series['base_tfidf_score']
    ],
    'Fo_exclusion': [
        mean_series['fo_emb_similarity'],
        mean_series['fo_rouge_score'],
        mean_series['fo_tfidf_score']
    ],
    'Ba_exclusion': [
        mean_series['ba_emb_similarity'],
        mean_series['ba_rouge_score'],
        mean_series['ba_tfidf_score']
    ]
}

table_df = pd.DataFrame(data, index=['Embedding', 'Rouge', 'Tfidf'])
table_df
table_df.to_csv("/ocean/projects/cis250068p/jhwang4/idl-project/notebooks/fine_exclusion_result.csv", index=True)